 # Week 9: Exercise 9.2
 File: DSC540_Paulovici_Exercise_9_2.py (.ipynb)<br>
 Name: Kevin Paulovici<br>
 Date: 2/9/2020<br>
 Course: DSC 540 Data Preparation (2203-1)<br>
 Assignment: Exercise 9.2

 In this exercise, you will create a twitter account (if you don’t already have one, or don’t wish to use your personal account) and practice pulling data from Twitter’s publicly available API. You can delete the twitter account as soon as you have completed the exercise. Include your code and output for each step.

 ## Part 1
 Create a Twitter API Key and Access Token (Data Wrangling with Python, pg. 365-366)

In [1]:
""" Retrieve Authentication keys from separate file 

To protect authentication keys, using a seprate file (keys.py) to 
store them, then calling the GET_KEYS function to use them
"""
# import keys

# API_KEY, API_SECRET, TOKEN_KEY, TOKEN_SECRET = keys.GET_KEYS()

# print("API_KEY len - {}\nAPI_SECRET len - {}\nTOKEN_KEY length - {}\nTOKEN_SECRET length - {}".format(
#     len(API_KEY), len(API_SECRET), len(TOKEN_KEY), len(TOKEN_SECRET)))

""" The above function should work but for some reason my import does not update
in the envirnment I'm working with. So, as alternative, just reading a file and setting the keys
format: API_KEY API_SECRET TOKEN_KEY TOKEN SECRET
"""
keys = []
with open('keys.txt') as fileIn:
    for line in fileIn:
        line = line.strip()
        line = line.split()

        for item in line:
            keys.append(item)

API_KEY = keys[0]
API_SECRET = keys[1]
TOKEN_KEY = keys[2]
TOKEN_SECRET = keys[3]


In [2]:
# input keys directly if no external file is used

# API_KEY = ""
# API_SECRET = ""
# TOKEN_KEY = ""
# TOKEN_SECRET = ""


 ## Part 2
 Do a single data pull from Twitter’s REST API (Data Wrangling with Python, pg. 366 – 368).

In [3]:
import oauth2
import json

# creat the OAuth connection
def oauth_req(url, key, secret, http_method="GET", post_body=b"",
                http_headers=None):
    
    consumer = oauth2.Consumer(key=API_KEY, secret=API_SECRET)
    token = oauth2.Token(key=key, secret=secret)
    client= oauth2.Client(consumer, token)
    resp, content = client.request(url, method=http_method,
                    body=post_body, headers=http_headers)
    
    return content

url = 'https://api.twitter.com/1.1/search/tweets.json?q=%23childlabor'
data = oauth_req(url, TOKEN_KEY, TOKEN_SECRET)

# save data to parse later
data = json.loads(data)
print(type(data))

with open('tweet_data.json', 'w') as data_file:
    json.dump(data, data_file)


<class 'dict'>


In [7]:
# show the json file wrote
import os

files = os.listdir()

if 'tweet_data.json' in files:
    print(True)

True


 ## Part 3
 Execute multiple queries at a time from Twitter’s REST API (Data Wrangling with Python, pg. 368 – 371).

In [4]:
import tweepy
import json
import dataset

def store_tweet(item):
    db = dataset.connect('sqlite:///data_wranfling.db')
    # table = db['tweets']
    table = db.create_table('tweets', primary_id=False)
    item_json = item._json.copy()

    for k, v in item_json.items():
        if isinstance(v, dict):
            item_json[k] = str(v)
    
    table.insert(item_json)

# set the OAuth and access token
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(TOKEN_KEY, TOKEN_SECRET)

api = tweepy.API(auth)

query = '#childlabor'
cursor = tweepy.Cursor(api.search, q=query, lanf="en")

# iterate through pages and save the data
for page in cursor.pages():
    for item in page:
        store_tweet(item)



In [5]:
# show the database wrote
import os

files = os.listdir()

if 'data_wranfling.db' in files:
    print(True)

True


 ## Part 4
 Do a data pull from Twitter’s Streaming API (Data Wrangling with Python, pg. 372 – 374).

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream

class Listener(StreamListener):
    def on_data(Self,data):
        print(data)
        return True

auth = OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(TOKEN_KEY, TOKEN_SECRET)

stream = Stream(auth, Listener())
stream.filter(track=['child labor'])


{"created_at":"Sun Feb 09 15:11:21 +0000 2020","id":1226523989905940480,"id_str":"1226523989905940480","text":"RT @marangdream: Feminists believe the exact opposite of this. Feminists recognize domestic labor as real labor, which merits compensation.\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":276602912,"id_str":"276602912","name":"Nomalili","screen_name":"Lili_ehm","location":"Cape Town, South Africa","url":null,"description":"Devil in a new dress | Selective Clown.","translator_type":"none","protected":false,"verified":false,"followers_count":1143,"friends_count":706,"listed_count":16,"favourites_count":6405,"statuses_count":43833,"created_at":"Sun Apr 03 18:04:49 +0000 2011","utc_offset":null,"time_zone":null,"geo_enabl

{"created_at":"Sun Feb 09 15:12:27 +0000 2020","id":1226524265467568129,"id_str":"1226524265467568129","text":"RT @marangdream: Feminists believe the exact opposite of this. Feminists recognize domestic labor as real labor, which merits compensation.\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":950826043119632384,"id_str":"950826043119632384","name":"Kumatetsu","screen_name":"WushuDonut","location":null,"url":"https:\/\/medium.com\/@pieinstomach?source=linkShare-6259415e44ff-1537124668","description":"This is the way","translator_type":"none","protected":false,"verified":false,"followers_count":287,"friends_count":385,"listed_count":1,"favourites_count":4271,"statuses_count":37060,"created_at":"Tue Jan 09 20:26:09 +0000 201

{"created_at":"Sun Feb 09 15:12:41 +0000 2020","id":1226524325072789505,"id_str":"1226524325072789505","text":"Nah scam them, it\u2019s free child labor for a big corporation","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":42327192,"id_str":"42327192","name":"The 17th of February","screen_name":"Crostiphex","location":null,"url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":186,"friends_count":705,"listed_count":0,"favourites_count":4330,"statuses_count":9013,"created_at":"Mon May 25 01:58:23 +0000 2009","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":null,"contributors_enabled":false,"is_translator":false,"profile_background_color":"C0DEED","profile_background_ima

{"created_at":"Sun Feb 09 15:13:09 +0000 2020","id":1226524440734896128,"id_str":"1226524440734896128","text":"RT @marangdream: Feminists believe the exact opposite of this. Feminists recognize domestic labor as real labor, which merits compensation.\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":493520787,"id_str":"493520787","name":"Lori harvey\u2019s look alike","screen_name":"Mss_dtt","location":"with ur local sheik x","url":"http:\/\/lifeofawestafricanbitch.tumblr.com","description":"fighting the patriarchy is exhausting.","translator_type":"regular","protected":false,"verified":false,"followers_count":1268,"friends_count":682,"listed_count":13,"favourites_count":158151,"statuses_count":110325,"created_at":"Wed Feb 15 21